DISCLAIMER: Isto não é, sob hipótese alguma, uma recomendação de investimento. <br>
A serventia destes algorítmos é apenas para didática. <br>

Desenvolvido em: <br>
Python 3.8.10 64-bit | <br>
Desenvolvido por: <br>
Fernando Luigi Cunha Dipp | <br>
Contato: <br>
fernando.dipp@gmail.com

In [1]:
# Bibliotecas necessárias
import requests as requests
from bs4 import BeautifulSoup

In [2]:
# Site que servirá como fonte para a obtenção dos dados
Site = 'https://opcoes.net.br/opcoes/pozinhos'
Pagina = requests.request('GET', Site)
# Retornando o Http Status da requisição acima
Pagina

<Response [200]>

In [3]:
# Fazendo o parse com o Beautifulsoup
Coleta = BeautifulSoup(Pagina.text, 'html.parser')
# Declarando e inicializando as variáveis da tabela de referência
TabelaBody = Coleta.find_all('table', attrs={'id': "tbl-opcoes-body"})

In [4]:
# Listando as 20 ações com mais liquidções nos últimos 2 meses (FONTE: https://fundamentus.com.br/resultado.php)
maisNegociadas= ('VALE','PETR','ELET','ITUB','BBDC','MGLU','PETR','BBAS','PRIO','RENT','B3SA','ABEV','GGBR','HAPV','LREN','SUZB','WEGE','JBSS','BPAC','AMER')

In [5]:
# listas para armazenagem dos dados de cada opção
opcao = []
tipo = []
vencimento = []
strike = []
distStrikePercent = []
ultimo = []
numNegocios = []
volFinanceiro = []
# dicionário para armazenar e organizar as listas
opcoesDict = {
'Ticker': opcao,
'Tipo' : tipo,
'Preco' : ultimo,
'Vencimento' : vencimento,
'Strike' : strike,
'Dist para Strike %' : distStrikePercent,
'Numero de Negocios' : numNegocios,
'Volume Financeiro' : volFinanceiro
}

In [6]:
for i in range(len(TabelaBody[0].find_all('tr'))): # Varrendo os dados da tabela iterando todos as linhas dela
    _opcao = TabelaBody[0].find_all('tr')[i].find_all('td')[0].text.strip() # Buscando nome da opção na tabela
    for j in range(len(maisNegociadas)): # Iterando cada item da lista de ações mais negociadas
        if maisNegociadas[j] in _opcao: # Comparando com o nome da empresa dona da opção
            # Adicionando a condição de filtro. No caso "> -10.0" está listando apenas as opções que estão entre 0% e -10% do seu respectivo strike
            _distStrikePercent = float(TabelaBody[0].find_all('tr')[i].find_all('td')[6].text.strip().replace(',','.'))
            if _distStrikePercent > -10.0:
                # Coletando as demais informações da opção encontrada
                _tipo = TabelaBody[0].find_all('tr')[i].find_all('td')[1].text.strip()
                _vencimento = TabelaBody[0].find_all('tr')[i].find_all('td')[2].text.strip()
                _strike = TabelaBody[0].find_all('tr')[i].find_all('td')[5].text.strip()
                _ultimo = TabelaBody[0].find_all('tr')[i].find_all('td')[10].text.strip()
                _numNegocios = TabelaBody[0].find_all('tr')[i].find_all('td')[15].text.strip()
                _volFinanceiro = TabelaBody[0].find_all('tr')[i].find_all('td')[16].text.strip()
                # Caso achado, imprime na tela
                print(_opcao + ' ' + str(_distStrikePercent) + '%'+' do strike, negociado a: R$' + _ultimo + ' vencendo em: ' + _vencimento)
                # Adicionando as informações nas suas respectivas listas
                opcao.append(_opcao)
                tipo.append(_tipo)
                vencimento.append(_vencimento)
                strike.append(_strike)
                distStrikePercent.append(_distStrikePercent)
                ultimo.append(_ultimo)
                numNegocios.append(_numNegocios)
                volFinanceiro.append(_volFinanceiro)


ABEVT137 -9.0% do strike, negociado a: R$0,02 vencendo em: 19/08/2022
ABEVT140 -7.4% do strike, negociado a: R$0,03 vencendo em: 19/08/2022
ABEVT142 -5.7% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
BBDCT145 -9.7% do strike, negociado a: R$0,02 vencendo em: 19/08/2022
BBDCT150 -6.5% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
BBDCT17 -9.0% do strike, negociado a: R$0,01 vencendo em: 19/08/2022
BBDCT181 -5.1% do strike, negociado a: R$0,05 vencendo em: 19/08/2022
BBDCT199 -6.4% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
ITUBT211 -5.9% do strike, negociado a: R$0,05 vencendo em: 19/08/2022
ITUBT218 -7.8% do strike, negociado a: R$0,05 vencendo em: 19/08/2022
ITUBT238 -9.7% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
ITUBT244 -8.8% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
ITUBT249 -6.9% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
WEGET280 -9.3% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
WEGET282 -8.5% do str

In [7]:
# Imprimindo na tela em forma de tabela html
import pandas as pd
TabelaOpcoes = pd.DataFrame(opcoesDict)
TabelaOpcoes.sort_values(by=['Dist para Strike %'], ascending=False)

,Ticker,Tipo,Preco,Vencimento,Strike,Dist para Strike %,Numero de Negocios,Volume Financeiro
6,BBDCT181,PUT,"0,05",19/08/2022,"18,16",-5.1,2.510,"28.457,00"
2,ABEVT142,PUT,"0,04",19/08/2022,"14,29",-5.7,101,"39.023,00"
8,ITUBT211,PUT,"0,05",19/08/2022,"24,82",-5.9,61,"24.626,00"
7,BBDCT199,PUT,"0,04",19/08/2022,"17,91",-6.4,2.361,"38.433,65"
4,BBDCT150,PUT,"0,04",19/08/2022,"14,82",-6.5,3,"67,00"
12,ITUBT249,PUT,"0,04",19/08/2022,"24,57",-6.9,37,"7.407,00"
1,ABEVT140,PUT,"0,03",19/08/2022,"14,04",-7.4,101,"12.896,00"
9,ITUBT218,PUT,"0,05",19/08/2022,"24,32",-7.8,9,"2.409,00"
14,WEGET282,PUT,"0,05",19/08/2022,"28,08",-8.5,1,"5,00"
11,ITUBT244,PUT,"0,04",19/08/2022,"24,07",-8.8,12,"3.432,00"


In [8]:
# Fazendo cálculos com as informações coletadas para cada opção
import math
disponivelInvest = 50
rentabAlvo = 100 #%
corretagem = 0
issCorretagem = 0 #%

for item in range(len(list(opcoesDict.values())[0])):
    precoOpcao = float(list(opcoesDict.values())[2][item].replace(',','.'))
    qntdMaxCompra = disponivelInvest / precoOpcao
    qntdMaxCompra = math.trunc(qntdMaxCompra / 100) * 100
    custoTotal = precoOpcao * qntdMaxCompra + (corretagem * (1+(issCorretagem/100)))
    breakEven = round((custoTotal + (corretagem * (1 + (issCorretagem / 100))))/qntdMaxCompra + 0.01, 2)
    stopGain = round((custoTotal + (corretagem * (1 + (issCorretagem / 100)))+((rentabAlvo / 100)*(precoOpcao*qntdMaxCompra)))/qntdMaxCompra, 2)
    lucroLiquido = round(stopGain*qntdMaxCompra-(custoTotal+(corretagem * (1+(issCorretagem/100)))),2)
    print('Possível comprar '+str(qntdMaxCompra)+' opções de '+ str(list(opcoesDict.values())[0][item]) + ' por R$'+str(list(opcoesDict.values())[2][item])+' cada, break-even aos R$' "%.2f" %breakEven +
    ', preço alvo aos R$'"%.2f" %stopGain+', tendo um lucro total de R$'"%.2f" %lucroLiquido)

Possível comprar 2500 opções de ABEVT137 por R$0,02 cada, break-even aos R$0.03, preço alvo aos R$0.04, tendo um lucro total de R$50.00
Possível comprar 1600 opções de ABEVT140 por R$0,03 cada, break-even aos R$0.04, preço alvo aos R$0.06, tendo um lucro total de R$48.00
Possível comprar 1200 opções de ABEVT142 por R$0,04 cada, break-even aos R$0.05, preço alvo aos R$0.08, tendo um lucro total de R$48.00
Possível comprar 2500 opções de BBDCT145 por R$0,02 cada, break-even aos R$0.03, preço alvo aos R$0.04, tendo um lucro total de R$50.00
Possível comprar 1200 opções de BBDCT150 por R$0,04 cada, break-even aos R$0.05, preço alvo aos R$0.08, tendo um lucro total de R$48.00
Possível comprar 5000 opções de BBDCT17 por R$0,01 cada, break-even aos R$0.02, preço alvo aos R$0.02, tendo um lucro total de R$50.00
Possível comprar 1000 opções de BBDCT181 por R$0,05 cada, break-even aos R$0.06, preço alvo aos R$0.10, tendo um lucro total de R$50.00
Possível comprar 1200 opções de BBDCT199 por R$0,

CÁLCULO DE BLACK-SCHOLES PARA CADA OPÇÃO ENCONTRADA

In [9]:
# Inicializando as listas e o dicionário de listas
ticker = []
strike = []
txJuros = []
dataRef = []
precoAcao = []
volImplicita = []
premioOpcao = []

BSDict = {
    'Data de Ref' : dataRef,
    'Ticker' : ticker,
    'Premio Opcao' : premioOpcao,
    'Strike' : strike,
    'Preco da Acao' : precoAcao,
    'Taxa de Juros' : txJuros,
    'Volatilidade Impl' : volImplicita
    }


In [10]:
# Coletando o resultado do cálculo de Black-Scholes para cada opção encontrada nos filtros acima
import time
for idx in range(len(list(opcoesDict.values())[0])):
    urlBlackScholes = 'https://opcoes.net.br/calculadora-Black-Scholes/' + list(opcoesDict.values())[0][idx]
    blackScholes = requests.get(urlBlackScholes)
    blackScholes = BeautifulSoup(blackScholes.text, 'html.parser')
    blackScholes = blackScholes.find_all('form', attrs={'id': "frmCalc"})
    blackScholes = blackScholes[0].find_all('input')
    ticker.append(blackScholes[1]['value'])
    strike.append(blackScholes[5]['value'])
    txJuros.append(blackScholes[7]['value'])
    dataRef.append(blackScholes[9]['value'])
    precoAcao.append(blackScholes[13]['value'])
    volImplicita.append(blackScholes[15]['value'])
    premioOpcao.append(blackScholes[17]['value'])
    time.sleep(1) # pausa de 1 segundo para evitar de o servidor do site bloquear o acesso por muitas requisições seguidas

In [11]:
# Imprimindo os resultados em formato de tabela html
import pandas as pd
BSTable = pd.DataFrame(BSDict)
BSTable.sort_values(by=['Premio Opcao'], ascending=False)

,Data de Ref,Ticker,Premio Opcao,Strike,Preco da Acao,Taxa de Juros,Volatilidade Impl
6,10/08/2022,BBDCT181,"0,05","18,16","19,13","13,75","28,53"
8,10/08/2022,ITUBT211,"0,05","24,82","26,38","13,75","29,41"
9,10/08/2022,ITUBT218,"0,05","24,32","26,38","13,75","34,38"
14,10/08/2022,WEGET282,"0,05","28,08","30,68","13,75","38,75"
2,10/08/2022,ABEVT142,"0,04","14,29","15,16","13,75","31,00"
4,10/08/2022,BBDCT150,"0,04","14,82","15,85","13,75","31,94"
7,10/08/2022,BBDCT199,"0,04","17,91","19,13","13,75","31,19"
10,10/08/2022,ITUBT238,"0,04","23,82","26,38","13,75","39,50"
11,10/08/2022,ITUBT244,"0,04","24,07","26,38","13,75","36,75"
12,10/08/2022,ITUBT249,"0,04","24,57","26,38","13,75","31,16"
